In [1]:
import pandas as pd
import scipy
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
ratings = pd.read_csv('/Users/junjie/Documents/RecommenderSystems-master/userDatas.csv')
movies = pd.read_csv('/Users/junjie/Documents/RecommenderSystems-master/netflix_titles.csv')
ratings = pd.merge(movies, ratings).drop(['date_added', 'release_year', 'rating', 'duration'], axis=1)
#ratings = pd.merge(movies, ratings)
ratings.head()

FileNotFoundError: [Errno 2] No such file or directory: '/Users/junjie/Documents/RecommenderSystems-master/userDatas.csv'

In [ ]:
ratings['cast'] = ratings['cast'].fillna(' ')
ratings['fav_cast'] = ratings['fav_cast'].fillna(' ')
ratings['director'] = ratings['director'].fillna(' ')
ratings['country'] = ratings['country'].fillna(' ')

In [ ]:
userRatings = ratings.pivot_table(index=['user_id'],columns=['title'],values='user_rating')
print("Before: ",userRatings.shape)
#userRatings = userRatings.dropna(how='all').fillna(0)
userRatings.fillna(0, inplace=True)
print("After: ",userRatings.shape)
userRatings.head(10)

In [ ]:
corrMatrix = userRatings.corr(method='pearson')
corrMatrix.head(10)

In [ ]:
def standardize(row):
    new_row = (row - row.mean())/(row.max()-row.min())
    return new_row

df_temp = userRatings
df_std = df_temp.apply(standardize)

user_similarity = cosine_similarity(df_std.T)
user_similarity

In [ ]:
from sklearn.metrics.pairwise import euclidean_distances
euclidean_dist = euclidean_distances(df_std.T)
euclidean_dist

In [ ]:
user_similarity_df = pd.DataFrame(user_similarity,index=userRatings.columns,columns=userRatings.columns)
user_similarity_df

In [ ]:
def get_similar_cosine(movie_name, user_rating):
    similar_score = user_similarity_df[movie_name]
    similar_score = similar_score.sort_values(ascending=False)
    #print(type(similar_ratings))
    return similar_score

In [ ]:
new_user1 = [("ONE PIECE",5),("Sword Art Online",2),("Sex Education",4),("Hunter X Hunter (2011)",5),("Attack on Titan",4)]
similar_movies = pd.DataFrame()
for movie,rating in new_user1:
    similar_movies = similar_movies.append(get_similar_cosine(movie,rating),ignore_index = True)

for (x,_) in new_user1:
    del similar_movies[x]
    #print("drop "+ x)
similar_movies.head()

In [ ]:
similar_movies.sum().sort_values(ascending=False).head(10)

In [ ]:
corrMatrix = userRatings.corr(method='pearson')
corrMatrix.head(10)

In [ ]:
def get_similar_pearson(movie_name, user_rating):
    similar_score = corrMatrix[movie_name]*(user_rating-2.5)
    similar_score = similar_score.sort_values(ascending=False)
    #print(type(similar_ratings))
    return similar_score

In [ ]:
new_user1 = [("ONE PIECE",5),("Sword Art Online",2),("Sex Education",4),("Hunter X Hunter (2011)",5),("Attack on Titan",4)]
similar_movies = pd.DataFrame()
for movie,rating in new_user1:
    similar_movies = similar_movies.append(get_similar_pearson(movie,rating),ignore_index = True)

for (x,_) in new_user1:
    del similar_movies[x]
    #print("drop "+ x)
similar_movies.head()

In [ ]:
similar_movies.sum().sort_values(ascending=False).head(10)

In [ ]:
action_lover = [("Amazing Spider-Man, The (2012)",5),("Mission: Impossible III (2006)",4),("Toy Story 3 (2010)",2),("2 Fast 2 Furious (Fast and the Furious 2, The) (2003)",4)]
similar_movies = pd.DataFrame()
for movie,rating in action_lover:
    similar_movies = similar_movies.append(get_similar(movie,rating),ignore_index = True)

similar_movies.head(10)
similar_movies.sum().sort_values(ascending=False).head(20)